In [1]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import json
from xml.etree.ElementTree import parse
import xmltodict

import pandas as pd
import numpy as np
import time
import os
import requests

from datetime import datetime

In [16]:
key = '0sZqTNtvkH3IKaiOWZVQnaAPbVzwwdjBmQpsFFByyGzhxYvMTHQTfHengAnJNjN9dJ3xSZBv+ZG++YmlUdM6nA=='
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
for i in range(1,11):
    params ={'serviceKey' : key, 
            'pageNo' : i, 
            'numOfRows' : '880', 
            'dataType' : 'JSON', 
            'dataCd' : 'ASOS', 
            'dateCd' : 'HR', 
            'startDt' : '20100101', 
            'startHh' : '01', 
            'endDt' : '20101231', 
            'endHh' : '01', 
            'stnIds' : '108' }

    response = requests.get(url, params=params)
    data = response.json()  # JSON 데이터 파싱

    # 파일로 저장
    with open(f"data{i}.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [15]:
data = response.json()  # JSON 데이터 파싱

# 파일로 저장
with open("data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [10]:
item = response.json()['response']['body']['items']['item']
df = pd.DataFrame()
for i in item:
    i = {k: [v] for k,v in i.items()}
    ddf = pd.DataFrame(i)
    df = pd.concat([df,ddf])
df

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,...,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
0,2010-01-01 01:00,1,108,서울,-11.0,0,,,1.2,0,...,,,,,-6.5,0,-4.7,-2.1,-0.6,0.6
0,2010-01-01 02:00,2,108,서울,-11.1,0,,,1.7,0,...,,,,,-6.5,0,-4.8,-2.1,-0.6,0.6
0,2010-01-01 03:00,3,108,서울,-12.1,0,,,1.6,0,...,,2000,16,,-6.7,0,-4.9,-2.2,-0.7,0.6
0,2010-01-01 04:00,4,108,서울,-12.3,0,,,1.4,0,...,,2000,,,-6.8,0,-5.0,-2.3,-0.7,0.6
0,2010-01-01 05:00,5,108,서울,-12.4,0,,,1.6,0,...,,2000,,,-7.0,0,-5.1,-2.4,-0.8,0.5
0,2010-01-01 06:00,6,108,서울,-12.4,0,,,1.6,0,...,,2000,,,-7.1,0,-5.2,-2.4,-0.8,0.5
0,2010-01-01 07:00,7,108,서울,-12.7,0,,,1.6,0,...,,2000,,,-7.2,0,-5.3,-2.5,-0.9,0.5
0,2010-01-01 08:00,8,108,서울,-12.6,0,,,1.5,0,...,,2300,,,-7.2,0,-5.4,-2.6,-0.9,0.5
0,2010-01-01 09:00,9,108,서울,-11.7,0,,,1.6,0,...,,2300,16,,-7.0,0,-5.3,-2.6,-1.0,0.5
0,2010-01-01 10:00,10,108,서울,-9.1,0,,,1.0,0,...,,2300,,,-6.3,0,-4.9,-2.5,-1.0,0.4


In [7]:
df['stnNm'].unique()[0]

'서울'

In [42]:
key = '0sZqTNtvkH3IKaiOWZVQnaAPbVzwwdjBmQpsFFByyGzhxYvMTHQTfHengAnJNjN9dJ3xSZBv%2BZG%2B%2BYmlUdM6nA%3D%3D'
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
# API 기본 정보
queryParams = '?' + urlencode({
        quote_plus('ServiceKey') : key,
        quote_plus('pageNo') : '1',   # 1 ~ 10
        quote_plus('numOfRows') : '40',  # 윤년때문에 대충 366으로 해둠
        quote_plus('dataType') : 'XML',
        quote_plus('dataCd') : 'ASOS',
        quote_plus('dateCd') : 'DAY',
        quote_plus('startDt') : '20200101',
        quote_plus('endDt') : '20200105',
        quote_plus('stnIds') : '108'})
request = urllib.request.Request(url + unquote(queryParams))

response_body = urlopen(request, timeout=60).read()

decode_data = response_body.decode('utf-8')

xml_parse = xmltodict.parse(decode_data)     # string인 xml 파싱
xml_dict = json.loads(json.dumps(xml_parse))
result = xml_dict['response']['header']['resultMsg']

# 응답 상태 코드 확인
# if response.status_code == 200:
#     try:
#         # JSON 형식으로 응답 파싱
#         data = response.json()
#         print(data)  # 필요한 데이터 추출 후 사용
#     except ValueError:
#         print("응답이 JSON 형식이 아닙니다.")
# else:
#     print(f"요청 실패: 상태 코드 {response.status_code}")

In [43]:
from pprint import pprint

pprint(xml_dict)
item = xml_dict['response']['body']['items']['item']

{'response': {'body': {'dataType': 'XML',
                       'items': {'item': [{'avgCm10Te': '0.5',
                                           'avgCm20Te': '1.9',
                                           'avgCm30Te': '2.9',
                                           'avgCm5Te': '-0.4',
                                           'avgLmac': '8.9',
                                           'avgM05Te': '4.1',
                                           'avgM10Te': '7.2',
                                           'avgM15Te': '9.8',
                                           'avgM30Te': '15.3',
                                           'avgM50Te': '17.0',
                                           'avgPa': '1021.1',
                                           'avgPs': '1032.1',
                                           'avgPv': '3.4',
                                           'avgRhm': '64.4',
                                           'avgTa': '-2.2',
                             

In [44]:
df = pd.DataFrame()
for i in item:
    i = {k: [v] for k,v in i.items()}
    ddf = pd.DataFrame(i)
    df = pd.concat([df,ddf])
df

,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
0,108,서울,2020-01-01,-2.2,-6.5,0001,0.3,1457,None,None,...,4.1,7.2,9.8,15.3,17.0,0.4,0.6,0.1,-{눈}-0230. {눈}0335-0440. {눈}0520-{눈}{강도0}0600-...,None
0,108,서울,2020-01-02,1.0,-0.7,0001,3.8,1500,None,None,...,3.7,7.0,9.7,15.2,17.0,0.7,1.0,None,-{연무}-{연무}{강도0}0300-{연무}{강도0}0600-{연무}{강도0}090...,None
0,108,서울,2020-01-03,-0.1,-3.4,0807,4.6,1547,None,None,...,3.6,6.8,9.6,15.1,16.9,1.4,1.9,None,{박무}0110-{박무}{강도1}0300-{박무}{강도1}0600-{박무}{강도0}...,None
0,108,서울,2020-01-04,1.2,-2.8,0512,6.1,1450,None,None,...,3.6,6.7,9.5,15.1,16.9,1.5,2.1,None,-{연무}-{연무}{강도0}0300-{연무}{강도0}0600-{연무}{강도0}090...,None
0,108,서울,2020-01-05,1.3,-3.2,0821,6.6,1453,None,None,...,3.5,6.5,9.3,15.0,16.9,1.5,2.1,None,{연무}0810-{연무}{강도0}0900-1140. {연무}1250-1320.,None
